# Setup

In [ ]:
# Import libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re # for cleanup
import spotipy
from config import *
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))

## Top 100

In [ ]:
# url used for top100
url = "https://www.billboard.com/charts/hot-100"

In [ ]:
# Download html
response = requests.get(url)
# 200 status code means OK! response.status_code
print(response.status_code)

In [ ]:
# Parse html
soup = BeautifulSoup(response.text, 'html.parser')
# Check up
# soup

In [ ]:
songnametext = soup.select("span.chart-element__information__song")[0].get_text()
songnametext

In [ ]:
Songnames = []
for elem in soup.select("span.chart-element__information__song"):
    Songnames.append(elem.get_text())


In [ ]:
artisttext = soup.select("span.chart-element__information__artist")[0].get_text()
artisttext

In [ ]:
Artists = []
for elem in soup.select("span.chart-element__information__artist"):
    Artists.append(elem.get_text())

In [ ]:
Top_100 = pd.DataFrame({'Songtitle': Songnames, 'Artist(s)': Artists})
Top_100

## Spotify

In [ ]:
path = '../Gnoosic/Spotify.csv'
Spotify =  pd.read_csv(path)
Spotify.head()

In [ ]:
len(Spotify)

### Clustering Spotify

In [ ]:
# Import libraries
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
# spotify df make new df (audio_df) only with the audio features? (only k to columns audiofeatures)
# audio_df.describe()
# -----making it all neat
# audio_df_prep = StandardScaler().fit_transform(audio_df) >>>gives me an array
# pd.DataFrame(audio_df_prep, columns=audio_df.columns).head()
# pd.DataFrame(audio_df_prep, columns=audio_df.columns).describe()
# -----clustering (the more the better)
# kmeans = KMeans(n_clusters=8, random_state=1234)  > random_state similar to seed
# kmeans.fit(audio_df_prep)
# clusters = kmeans.predict(audio_df_prep)
# pd.Series(clusters).value_counts().sort_index()
# -----which song was assigned which?
# Spotify_df = pd.DataFrame(Spotify)
# Spotify_df["cluster"] = clusters
# Spotify_df.head()

In [ ]:
audio_df = Spotify.filter(['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo'])

In [ ]:
audio_df.describe()

#### making the data neat with StandarsScaler

In [ ]:
scaler = StandardScaler()
audio_df_prep = scaler.fit_transform(audio_df)

In [ ]:
pd.DataFrame(audio_df_prep, columns=audio_df.columns).head()

In [ ]:
pd.DataFrame(audio_df_prep, columns=audio_df.columns).describe()

#### finding the best cluster

In [ ]:
K = range(2, 40)
inertia = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(audio_df_prep)
    inertia.append(kmeans.inertia_)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')

In [ ]:
K = range(2, 40)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(audio_df_prep)
    silhouette.append(silhouette_score(audio_df_prep, kmeans.predict(audio_df_prep)))


plt.figure(figsize=(16,8))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Silhouette Method showing the optimal k')

In [ ]:
#Choosing 25

#### Clustering

In [ ]:
kmeans = KMeans(n_clusters=25, random_state=1234)

In [ ]:
kmeans.fit(audio_df_prep)

In [ ]:
clusters = kmeans.predict(audio_df_prep)

In [ ]:
pd.Series(clusters).value_counts().sort_index()

#### Check cluster assignments?

In [ ]:
Spotify_df = pd.DataFrame(Spotify)

In [ ]:
Spotify_df["cluster"] = clusters

In [ ]:
Spotify_df.head()

# Prototype

Pseudocode:

User inputs song (not lower/uppercase sensitive)

Is the song in the Top 100?
    
    YES: Recommend another Song from the Top 100 list
    NO: as k for artist name then Match the song to one of the clusters and recommend a song from that cluster. if there is an index error try and except
    
note prototype 2: input matching cluster > get input-cluster#, recommend song with same cluster#

1)user input song call to spotify api to get audio features (combined search with artist and song?) get uri > get audio feautures

2)standart scaler to transform the feautures audio features k.means.predict cluster, then match

        

In [ ]:
# Songinput (not lower/uppercase sensitive)
# Songinput = input("To teach Gnod what you are like, please type in 1 Song that you already know and like: ").lower()

In [ ]:
# FUNCTIONS Notes

# Check if it is in the top 100
# Top_100.loc[Top_100['Songtitle'].str.lower() == Songinput]

# how to get Match
# Match = (Top_100.loc[Top_100['Songtitle'].str.lower() == Songinput])
# print(Match) > if its's a match it will show 1
# Match.shape[0] > so this will be 1 also ergo if Match.shape[0] is not 1 it is not a match

# Songrecommendation Top 100
    
def song_recommendation():
    Recommendation = Top_100.sample()
    return print("Songrecommendation: " + Recommendation.iloc[0,0] + " by " + Recommendation.iloc[0,1])


# Recommendation all

def Recommendation():
    Songinput  = input("To teach Gnod what you are like, please type in 1 Song that you already know and like: ").lower()
    Match = (Top_100.loc[Top_100['Songtitle'].str.lower() == Songinput])
    if Match.shape[0] == 1:
        song_recommendation()
    else:
        artist = input("Please also give us the artist of that song: ")
        try:
            result = sp.search(q="track:"+Songinput+" artist:"+artist,limit=1) #> get a json objects
            my_dict = sp.audio_features(result["tracks"]["items"][0]["uri"])[0]
            my_dict_2 = { key: [my_dict[key]] for key in list(my_dict.keys()) }
            df_1 = pd.DataFrame(my_dict_2)
            df_2 = df_1.drop(['type', 'id', 'uri', 'track_href','analysis_url', 'duration_ms', 'time_signature'], axis=1)
            df_2_prep = scaler.transform(df_2)
            clusters_1 = kmeans.predict(df_2_prep)
            recomm = Spotify_df[Spotify_df['cluster'] == clusters_1[0]].sample()
            print('Songrecommendation: ' + recomm.iloc[0,0] + " by " + recomm.iloc[0,1])
        except IndexError:
            print('Sorry, this song seems to be not existent.')


# App

In [ ]:
Recommendation()

In [ ]:
Recommendation()